# SSl/TSL project

## install libraries

## generate certificate

In [1]:
import os

# generate key file
generate_key_command = "openssl genpkey -algorithm RSA -out server_key.pem -aes256"
# generate certificate request
generate_certificate_signing_request_command = "openssl req -new -key server_key.pem -out server_csr.pem"
# geenrate seld-signed certificate
generate_self_signed_certificate_command = "openssl x509 -req -days 365 -in server_csr.pem -signkey server_key.pem -out server_cert.pem"


# os.system(generate_key_command)
# os.system(generate_certificate_signing_request_command)
# os.system(generate_self_signed_certificate_command)

## check the certificates

In [2]:
from cryptography import x509
from cryptography.hazmat.backends import default_backend

def read_csr(file_path):
    with open(file_path, "rb") as f:
        csr_data = f.read()
    csr = x509.load_pem_x509_csr(csr_data, default_backend())
    
    print("Subject:", csr.subject)

    print("Signature Algorithm:", csr.signature_algorithm_oid)
    print("Public Key:", csr.public_key())
    
    for attribute in csr.subject:
        print(f"{attribute.oid._name}: {attribute.value}")

read_csr("../../server_csr.pem")

Subject: <Name(C=IR,ST=Tehran,L=Tehran,O=Amirkabir,OU=CE,CN=Sayed,1.2.840.113549.1.9.1=esmirk.137@gmail.com)>
Signature Algorithm: <ObjectIdentifier(oid=1.2.840.113549.1.1.11, name=sha256WithRSAEncryption)>
Public Key: <cryptography.hazmat.bindings._rust.openssl.rsa.RSAPublicKey object at 0x72db4678aa90>
countryName: IR
stateOrProvinceName: Tehran
localityName: Tehran
organizationName: Amirkabir
organizationalUnitName: CE
commonName: Sayed
emailAddress: esmirk.137@gmail.com


## implement analyzer

In [3]:
import pyshark
import threading

def capture_tls_packets():
    capture = pyshark.LiveCapture(interface='lo', bpf_filter='tcp port 8443')
    print("Capturing packets...")

    for packet in capture.sniff_continuously(packet_count=10):
        if 'TLS' in packet:
            print("TLS Packet:")
            print(f"Version: {packet.tls.record_version}")
            print(f"Cipher: {packet.tls.record_cipher_suites}")
            print(f"Length: {packet.tls.record_length}")

## Run Server

In [4]:
from ssl_server import start_ssl_server

## Run Client

## Run Analyzer

In [ ]:
# Run the capture function in a separate thread
capture_thread = threading.Thread(target=capture_tls_packets)
capture_thread.start()
capture_thread.join()